In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from rouge import Rouge
import torch
import pandas as pd

/Users/inigoparra/opt/anaconda3/envs/TF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-21 18:55:15.103159: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
model_name = 'google/flan-t5-base'

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Downloading: 100%|██████████| 2.54k/2.54k [00:00<00:00, 1.22MB/s]
Downloading: 100%|██████████| 792k/792k [00:00<00:00, 7.76MB/s]
Downloading: 100%|██████████| 2.42M/2.42M [00:00<00:00, 4.08MB/s]
Downloading: 100%|██████████| 2.20k/2.20k [00:00<00:00, 914kB/s]
Downloading: 100%|██████████| 674/674 [00:00<00:00, 266kB/s]
Downloading: 100%|██████████| 50.8k/50.8k [00:00<00:00, 531kB/s]
Downloading: 100%|██████████| 9.45G/9.45G [03:27<00:00, 45.6MB/s] 
Downloading: 100%|██████████| 9.60G/9.60G [06:49<00:00, 23.5MB/s] 
Downloading: 100%|██████████| 9.96G/9.96G [03:47<00:00, 43.8MB/s] 
Downloading:  35%|███▍      | 3.49G/10.0G [00:47<01:26, 74.9MB/s]

KeyboardInterrupt: 

Downloading:  35%|███▍      | 3.50G/10.0G [01:02<01:26, 74.9MB/s]

In [6]:
def get_response_and_scores(prompt, reference):
    # Encode & Decode
    inputs = tokenizer.encode("Reply to this prompt: " + prompt, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=150, num_return_sequences=1, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)

    input_ids = tokenizer.encode(response, return_tensors='pt')

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss
    perplexity = torch.exp(loss)

    rouge = Rouge()
    scores = rouge.get_scores(response, reference)

    df = pd.DataFrame({
        'Metric': ['ROUGE-1', 'ROUGE-2', 'ROUGE-L'],
        'Recall': [scores[0]['rouge-1']['r'], scores[0]['rouge-2']['r'], scores[0]['rouge-l']['r']],
        'Precision': [scores[0]['rouge-1']['p'], scores[0]['rouge-2']['p'], scores[0]['rouge-l']['p']],
        'F1 Score': [scores[0]['rouge-1']['f'], scores[0]['rouge-2']['f'], scores[0]['rouge-l']['f']],
    })

    print(f'ROUGE scores\n{df}')

    return response, perplexity.item(), scores

In [7]:
new_prompt = 'Imagine you are an expert on AI. Explain the impact of AI.'
reference_text = 'AI has a significant impact on various fields, including healthcare, transportation, and finance. It can automate tasks, improve decision-making, and provide new insights from data.'

response, perplexity, rouge_scores = get_response_and_scores(new_prompt, reference_text)
print('\nResponse:', response)
print('Perplexity:', perplexity)


ROUGE scores
    Metric  Recall  Precision  F1 Score
0  ROUGE-1     0.0        0.0       0.0
1  ROUGE-2     0.0        0.0       0.0
2  ROUGE-L     0.0        0.0       0.0

Response: <pad><extra_id_0>: Reply to this prompt: Reply to this prompt:</s>
Perplexity: 16.69925880432129
